In [1]:
_RIOT_API_KEY='RGAPI-0a48f46d-50e5-4744-b116-3aa0a270934c'
from riotwatcher import LolWatcher
import helper_functions as help
import constants
import pandas as pd
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
import concurrent.futures

lol_watcher = LolWatcher(_RIOT_API_KEY)


1. get puuid for a summoner name - Summonerv4/by-name/summonername
2. get list of match ids by puuid - Matchv5/ids
3. Check database for which match ids i don't have - find_to_slurp
4. Create a function that does the following
- get match by match id - Matchv5/matchids
    - Store in a DB
- get match timeline by match id
    - store in a DB
5. Create concurrent processing that will speed this up

# Find To Slurp


In [3]:

# user = 'root'
# password = 'iamgroot482'
# port = '3306'
# host = '127.0.0.1'
# database = 'riot_api'
# url="mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}".format(
#             user, password, host, port, database
#         )
# engine = create_engine(url)    
engine = help.create_mysql_engine()
to_slurp = help.find_to_slurp('souporsecret', lol_watcher, 'lol_match_details', engine)
to_slurp

['NA1_4393904069']

In [2]:
## This is the mother function you run.
## summoner_name  (string) - Name of summoner to pull data for
## lol_watcher (object) - Authenticated lol_watcher object
## table_name (string) - Table we want to populate. either 'lol_match_timeline' or 'lol_match_details'
## db_engine (sqlAlchemy Engine Object)- Connection to mySQLDatabase from sqlAlchemy's create_engine.
##                                        Use create_mysql_engine helper function with default settings.
## region (string) - Region account is in. Defaults to NA
def collect_riot_api_data(summoner_name, lol_watcher, table_name, db_engine, region='na1', num_worker_threads=4, match_upload_limit=20):
    # First We run find_to_slurp to avoid pulling data for matches we have already pulled
    to_slurp = help.find_to_slurp(summoner_name, lol_watcher, table_name, db_engine)

    # Next we use slurp_data to pull the data from Riot API
    # Set Up multiprocessing for this later

    def worker (match_id):
        df_to_upload = help.slurp_data(lol_watcher, match_id, table_name, region=region)
        if df_to_upload is not None:
            df_to_upload.to_sql(con=db_engine, name=table_name, if_exists='append', index = False)
            print(f"Uploaded match id {match_id} {table_name} data.")
        else: 
            print(f'Match id {match_id} returns None.')

    with concurrent.futures.ThreadPoolExecutor(max_workers = num_worker_threads) as executor:
        to_slurp = to_slurp[:match_upload_limit]
        futures = [ executor.submit(worker, match_id) for match_id in to_slurp ]

        try: 
            for future in concurrent.futures.as_completed(futures):
                future.result()
        except Exception as e:
            print(f"Exception {e} thrown. Cancelling futures.")

    # for match_id in to_slurp[:10]:
    #     df_to_upload = help.slurp_data(lol_watcher, match_id, table_name, region='na1')
    #     df_to_upload.to_sql(con=db_engine, name=table_name, if_exists='append', index = False)
    #     print(f"Uploaded match_id {match_id} {table_name} data.")
    # print('Done')

In [5]:
engine = help.create_mysql_engine()
collect_riot_api_data('souporsecret', lol_watcher, 'lol_match_timeline', engine, match_upload_limit=800, num_worker_threads=1)

Empty Game. Returning None for Match Timeline.
Match id NA1_4393904069 returns None.


In [6]:
engine = help.create_mysql_engine()
collect_riot_api_data('souporsecret', lol_watcher, 'lol_match_details', engine, match_upload_limit=800, num_worker_threads=1)

Empty Game. Returning None for Match Details.
Match id NA1_4393904069 returns None.
